<a href="https://colab.research.google.com/github/AlexanderPiavis/GSB544/blob/main/Practice_Activity_4_1_Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [2]:
# YOUR CODE HERE
import requests
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [3]:
# YOUR CODE HERE
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [3]:
print(soup.title)

<title>List of United States cities by population - Wikipedia</title>


To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [4]:
print(soup.select_one("table"))

<table class="sidebar nomobile nowraplinks"><tbody><tr><th class="sidebar-title">Population tables<br/>of U.S. cities</th></tr><tr><td class="sidebar-image"><span class="mw-default-size mw-image-border" typeof="mw:File/Frameless"><a class="mw-file-description" href="/wiki/File:View_of_Empire_State_Building_from_Rockefeller_Center_New_York_City_dllu_(cropped).jpg"><img alt="New York City skyline" class="mw-file-element" data-file-height="4616" data-file-width="8207" decoding="async" height="169" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/7a/View_of_Empire_State_Building_from_Rockefeller_Center_New_York_City_dllu_%28cropped%29.jpg/330px-View_of_Empire_State_Building_from_Rockefeller_Center_New_York_City_dllu_%28cropped%29.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/7a/View_of_Empire_State_Building_from_Rockefeller_Center_New_York_City_dllu_%28cropped%29.jpg/500px-View_of_Empire_State_Building_from_Rockefeller_Center_New_York_City_dllu_%28cropped%29.jpg 1.5x, 

We can use attributes like `class=` and `style=` to narrow down the list.

In [6]:
len(soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  ))

1

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [4]:
table = soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [7]:
rows = []

#iterate over all rows in the cities population table
for city in table.find_all("tr")[2:]:  #This could also be [2:]
  #Get all the cells in the row
  cells = city.find_all("td")

  #Find information
  city_tag = cells[0].find("a") or cells[0]
  city_name = city_tag.text

  state_tag = cells[1].find("a") or cells[1]
  state_name = state_tag.text

  estimate_2024 = cells[2]
  estimate_2024 = estimate_2024.get_text(strip = True)

  census_2020 = cells[3]
  census_2020 = census_2020.get_text(strip = True)

  change_tag = cells[4].find("span", attrs={
      "style": lambda s: s and "color" in s
  })
  change = change_tag

  land_area_mi_squared_tag = cells[5]
  land_area_mi_squared = land_area_mi_squared_tag.get_text(strip = True)

  land_area_km_squared_tag = cells[6]
  land_area_km_squared = land_area_km_squared_tag.get_text(strip = True)

  density_mi_squared_tag = cells[7]
  density_mi_squared = density_mi_squared_tag.get_text(strip = True)

  density_km_squared_tag = cells[8]
  density_km_squared = land_area_km_squared_tag.get_text(strip = True)

  location_tag = cells[9].find("span", class_="geo-dec")
  location = location_tag.text

  #Append Data
  rows.append({
      "City": city_name,
      "State": state_name,
      "Estimate 2024": estimate_2024,
      "Census 2020": census_2020,
      "Change": change,
      "Land Area (mi^2)": land_area_mi_squared,
      "Land Area (km^2)": land_area_km_squared,
      "Density (mi^2)": density_mi_squared,
      "Density (km^2)": density_km_squared,
      "Location": location
  })

In [8]:
cities = pd.DataFrame(rows)
cities = cities.sort_values(by="City")
cities

,City,State,Estimate 2024,Census 2020,Change,Land Area (mi^2),Land Area (km^2),Density (mi^2),Density (km^2),Location
223,Abilene,TX,"130,501","125,182",[+4.25%],106.7,276.4,"1,173",276.4,32.45°N 99.74°W
139,Akron,OH,"189,664","190,469",[−0.42%],61.9,160.3,"3,077",160.3,41.08°N 81.52°W
338,Albany,NY,"101,317","99,224",[+2.11%],21.4,55.4,"4,637",55.4,42.67°N 73.80°W
31,Albuquerque,NM,"560,326","564,559",[−0.75%],187.3,485.1,"3,014",485.1,35.10°N 106.65°W
167,Alexandria,VA,"159,102","159,467",[−0.23%],14.9,38.6,"10,702",38.6,38.82°N 77.08°W
...,...,...,...,...,...,...,...,...,...,...
90,Winston-Salem,NC,"255,769","249,545",[+2.49%],132.7,343.7,"1,881",343.7,36.10°N 80.26°W
311,Woodbridge,NJ,"106,101","103,639",[+2.38%],23.3,60.3,"4,448",60.3,40.56°N 74.29°W
113,Worcester,MA,"211,286","206,518",[+2.31%],37.4,96.9,"5,522",96.9,42.27°N 71.81°W
114,Yonkers,NY,"211,040","211,569",[−0.25%],18.0,46.6,"11,754",46.6,40.95°N 73.87°W


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



Skip

The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [27]:
#pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

HTTPError: HTTP Error 403: Forbidden

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [28]:
#pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

HTTPError: HTTP Error 403: Forbidden

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [29]:
#df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
#df_cities2

HTTPError: HTTP Error 403: Forbidden

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [30]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [9]:
URL = "https://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

In [10]:
soup = BeautifulSoup(response.content, "html.parser")

In [15]:
classes = soup.find_all("div", class_="courseblock")

In [27]:
print(len(classes))

74


In [32]:
import re
rows = []
for i in range(0, 74, 1):
  name = classes[i].find("p", class_="courseblocktitle").get_text(strip = True)
  name = name.replace("\xa0", " ")
  name = re.sub(r"\.\s*\d+(\s*–\s*\d+)?\s*units?$", "", name) #Got this from AI

  term = classes[i].find("p", class_="noindent").get_text(strip = True)
  term = term.replace("Term Typically Offered:", "").strip()

  rows.append({
      "Course Name": name,
      "Term": term
  })


In [34]:
 CalPoly_DATA_STAT_Courses = pd.DataFrame(rows)


In [35]:
CalPoly_DATA_STAT_Courses

,Course Name,Term
0,DATA 100. Data Science for All I,"F, W, SP"
1,DATA 301. Introduction to Data Science,"F, W, SP"
2,DATA 401. Data Science Process and Ethics,F
3,DATA 402. Mathematical Foundations of Data Sci...,F
4,DATA 403. Data Science Projects Laboratory,F
...,...,...
69,STAT 551. Statistical Learning with R,F
70,STAT 566. Graduate Consulting Practicum,SP
71,STAT 570. Selected Advanced Topics.1-4 units,TBD
72,STAT 590. Graduate Seminar in Statistics,"F, W, SP"


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag